## XYZ Pro Features
This notebook demonstrates some of the pro features for XYZ Hub API.

XYZ paid features can be found here: [xyz pro features](https://www.here.xyz/xyz_pro/).

XYZ plans can be found here: [xyz plans](https://developer.here.com/pricing).

### Virtual Space
A virtual space is described by definition which references other existing spaces(the upstream spaces).
Queries being done to a virtual space will return the features of its upstream spaces combined.

Below are different predefined operations of how to combine the features of the upstream spaces.

- [group](#group_cell)
- [merge](#merge_cell)
- [override](#override_cell)
- [custom](#custom_cell)

In [ ]:
# Make necessary imports.
import os
import json
import warnings

from xyzspaces.datasets import get_chicago_parks_data, get_countries_data
from xyzspaces.exceptions import ApiError
import xyzspaces

<div class="alert alert-block alert-warning">
<b>Warning:</b> Before running below cells please make sure you have XYZ Token to interact with xyzspaces. 
                Please see README.md in notebooks folder for more info on XYZ_TOKEN
</div>

In [ ]:
# Make a XYZ object
try:
    xyz_token = os.environ["XYZ_TOKEN"]
except KeyError:
    xyz_token = "MY-FANCY-XYZ-TOKEN"
    if xyz_token == "MY-FANCY-XYZ-TOKEN":
        warnings.warn(
            "Please either set your actual token to env variable XYZ_TOKEN or "
            "just assign value of your actual token to variable xyz_token above."
        )
xyz = xyzspaces.XYZ(credentials=xyz_token)

In [ ]:
# create two spaces which will act as upstream spaces for virtual space created later.

title1 = "Testing xyzspaces"
description1 = "Temporary space containing countries data."
space1 = xyz.spaces.new(title=title1, description=description1)

# Add some data to it space1
gj_countries = get_countries_data()
space1.add_features(features=gj_countries)
space_id1 = space1.info["id"]

title2 = "Testing xyzspaces"
description2 = "Temporary space containing chicago parks data."
space2 = xyz.spaces.new(title=title2, description=description2)

# Add some data to space2
with open('./data/chicago_parks.geo.json', encoding="utf-8-sig") as json_file:
    gj_chicago = json.load(json_file)
    
space2.add_features(features=gj_chicago)
space_id2 = space2.info["id"]

<a id='group_cell'></a>
#### Group
Group means to combine the content of the specified spaces. All objects of each space will be part of the response when the virtual space is queried by the user. The information about which object came from which space can be found in the XYZ-namespace in the properties of each feature. When writing back these objects to the virtual space they'll be written back to the upstream space from which they were actually coming.

In [ ]:
# Create a new virtual space by grouping two spaces created above.

title = "Virtual Space for coutries and chicago parks data"
description = "Test group functionality of virtual space"

upstream_spaces = [space_id1, space_id2]
kwargs = {"virtualspace": dict(group=upstream_spaces)}

vspace = xyz.spaces.virtual(title=title, description=description, **kwargs)
print(json.dumps(vspace.info, indent=2))

In [ ]:
# Reading a particular feature from space1 via virtual space.

vfeature1 = vspace.get_feature(feature_id="FRA")
feature1 = space1.get_feature(feature_id="FRA")
assert vfeature1 == feature1

# Reading a particular feature from space2 via virtual space.
vfeature2 = vspace.get_feature(feature_id="LP")
feature2 = space2.get_feature(feature_id="LP")
assert vfeature2 == feature2

In [ ]:
# Deleting a feature from virtual space deletes corresponding feature from upstream space.

vspace.delete_feature(feature_id="FRA")
try:
    space1.get_feature("FRA")
except ApiError as err:
    print(err)


In [ ]:
# Delete temporary spaces created.
vspace.delete()
space1.delete()
space2.delete()

<a id='merge_cell'></a>
#### Merge
Merge means that objects with the same ID will be merged together. If there are duplicate feature-IDs in the various data of the upstream spaces, the duplicates will be merged to build a single feature. The result will be a response that is guaranteed to have no features with duplicate IDs. The merge will happen in the order of the space-references in the specified array. That means objects coming from the second space will overwrite potentially existing property values of objects coming from the first space. The information about which object came from which space(s) can be found in the XYZ-namespace in the properties of each feature. When writing back these objects to the virtual space they'll be written back to the upstream space from which they were actually coming, or the last one in the list if none was specified.When deleting features from the virtual space a new pseudo-deleted feature is written to the last space in the list. Trying to read the feature with that ID from the virtual space is not possible afterward.

In [ ]:
# create two spaces with duplicate data 

title1 = "Testing xyzspaces"
description1 = "Temporary space containing countries data."
space1 = xyz.spaces.new(title=title1, description=description1)

with open('./data/chicago_parks.geo.json', encoding="utf-8-sig") as json_file:
    gj_chicago = json.load(json_file)
    
# Add some data to it space1
space1.add_features(features=gj_chicago)
space_id1 = space1.info["id"]

title2 = "Testing xyzspaces duplicate"
description2 = "Temporary space containing countries data duplicate"
space2 = xyz.spaces.new(title=title1, description=description1)

# Add some data to it space2
space2.add_features(features=gj_chicago)
space_id2 = space2.info["id"]

In [ ]:
# update a particular feature of second space so that post merge virtual space will have this feature merged
lp = space2.get_feature("LP")
space2.update_feature(feature_id="LP", data=lp, add_tags=["foo", "bar"])

In [ ]:
# Create a new virtual space by merging two spaces created above.

title = "Virtual Space for coutries and chicago parks data"
description = "Test merge functionality of virtual space"

upstream_spaces = [space_id1, space_id2]
kwargs = {"virtualspace": dict(merge=upstream_spaces)}

vspace = xyz.spaces.virtual(title=title, description=description, **kwargs)
print(vspace.info)

In [ ]:
vfeature1 = vspace.get_feature(feature_id="LP")
assert vfeature1["properties"]["@ns:com:here:xyz"]["tags"] == ["foo", "bar"]

In [ ]:
bp = space2.get_feature("BP")
space2.update_feature(feature_id="BP", data=lp, add_tags=["foo1", "bar1"])

In [ ]:
vfeature2 = vspace.get_feature(feature_id="BP")
assert vfeature2["properties"]["@ns:com:here:xyz"]["tags"] == ["foo1", "bar1"]

In [ ]:
space1.delete()
space2.delete()
vspace.delete()

<a id='override_cell'></a>
#### Override
Override means that objects with the same ID will be overridden completely. If there are duplicate feature-IDs in the various data of the upstream spaces, the duplicates will be overridden to result in a single feature. The result will be a response that is guaranteed to have no features with duplicate IDs. The override will happen in the order of the space-references in the specified array. That means objects coming from the second space one will override potentially existing features coming from the first space. The information about which object came from which space can be found in the XYZ-namespace in the properties of each feature. When writing back these objects to the virtual space they'll be written back to the upstream space from which they were actually coming. When deleting features from the virtual space the same rules as for merge apply.

In [ ]:
# create two spaces with duplicate data 

title1 = "Testing xyzspaces"
description1 = "Temporary space containing countries data."
space1 = xyz.spaces.new(title=title1, description=description1)

with open('./data/chicago_parks.geo.json', encoding="utf-8-sig") as json_file:
    gj_chicago = json.load(json_file)
    
# Add some data to it space1
space1.add_features(features=gj_chicago)
space_id1 = space1.info["id"]

title2 = "Testing xyzspaces duplicate"
description2 = "Temporary space containing countries data duplicate"
space2 = xyz.spaces.new(title=title1, description=description1)

# Add some data to it space2
space2.add_features(features=gj_chicago)
space_id2 = space2.info["id"]

In [ ]:
# Create a new virtual space by override operation.

title = "Virtual Space for coutries and chicago parks data"
description = "Test merge functionality of virtual space"

upstream_spaces = [space_id1, space_id2]
kwargs = {"virtualspace": dict(override=upstream_spaces)}

vspace = xyz.spaces.virtual(title=title, description=description, **kwargs)
print(vspace.info)

In [ ]:
bp = space2.get_feature("BP")
space2.update_feature(feature_id="BP", data=bp, add_tags=["foo1", "bar1"])

In [ ]:
vfeature2 = vspace.get_feature(feature_id="BP")
assert vfeature2["properties"]["@ns:com:here:xyz"]["tags"] == ["foo1", "bar1"]

In [ ]:
space1.delete()
space2.delete()
vspace.delete()

<a id='custom_cell'></a>
#### Custom

In [ ]:
# TODO: Add custom example
# Custom operation needs special permissions via CMEKB JIRA, add example once permissions are granted.

### Applying clustering in space

In [ ]:
# create two spaces which will act as upstream spaces for virtual space created later.

title1 = "Testing xyzspaces"
description1 = "Temporary space containing countries data."
space1 = xyz.spaces.new(title=title1, description=description1)

# Add some data to it space1
gj_countries = get_countries_data()
space1.add_features(features=gj_countries)
space_id1 = space1.info["id"]

In [ ]:
# Genereate clustering for the space
space1.cluster(clustering='hexbin')


In [ ]:
# Delete created space
space1.delete()

### Rule based Tagging
Rule based tagging makes tagging multiple features in space tagged to a particular tag, based in rules mentioned based on JSON-path expression. Users can update space with a map of rules where the key is the tag to be applied to all features matching the JSON-path expression being the value.

If multiple rules are matching, multiple tags will be applied to the according to matched sets of features. It could even happen that a feature is matched by multiple rules and thus multiple tags will get added to it.

In [ ]:
# Create a new space
title = "Testing xyzspaces"
description = "Temporary space containing chicago parks data."
space = xyz.spaces.new(title=title, description=description)

In [ ]:
# Add data to the space.
with open('./data/chicago_parks.geo.json', encoding="utf-8-sig") as json_file:
    gj_chicago = json.load(json_file)
_ = space.add_features(features=gj_chicago)

In [ ]:
# update space to add tagging rules to the above mentioned space.
tagging_rules = {
        "large": "$.features[?(@.properties.area>=500)]",
        "small": "$.features[?(@.properties.area<500)]",
    }
_ = space.update(tagging_rules=tagging_rules)

In [ ]:
# verify that features are tagged correctly based on rules.
large_parks = space.search(tags=["large"])
for park in large_parks:
    assert park["id"] in ["LP", "BP", "JP"]
small_parks = space.search(tags=["small"])
for park in small_parks:
    assert park["id"] in ["MP", "GP", "HP", "DP", "CP", "COP"]

In [ ]:
# Delete created space
space.delete()

### Activity Log
The Activity log  will enable tracking of changes in your space.
To activate it, just create a space with the listener added and enable_uuid set to True.
More information on the activity log can be found [here](https://www.here.xyz/api/devguide/activitylogguide/).

In [ ]:
title = "Activity-Log Test"
description = "Activity-Log Test"
listeners = {
    "id": "activity-log",
    "params": {"states": 5, "storageMode": "DIFF_ONLY", "writeInvalidatedAt": "true"},
    "eventTypes": ["ModifySpaceEvent.request"],
}
space = xyz.spaces.new(
    title=title, description=description, enable_uuid=True, listeners=listeners,
)


In [ ]:
from time import sleep

# As activity log is async operation adding sleep to get info
sleep(5)
print(json.dumps(space.info, indent=2))

In [ ]:
space.delete()